In [ ]:
%load_ext autoreload
%autoreload 2

import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import xgboost as xgb
import random 
import os
import ujson

from typing import List, Dict, Tuple, Union, Any

def set_seed():
    np.random.seed(42)
    random.seed(42)
    
set_seed()

In [ ]:
icx_df = pd.read_parquet("out/icx.parquet")
clx_df = pd.read_parquet("out/clx.parquet")

icx_ssj = ujson.load(open("out/ssj.icx.json"))
clx_ssj = ujson.load(open("out/ssj.clx.json"))

In [ ]:
def pivot_df(df: pd.DataFrame):
    df = df[
        [
            "timestamp",
            "%cpu_lc0",
            "%cpu_lc1",
            "cpu_lc0",
            "cpu_lc1",
            "phy_cpu",
            "name",
            "%cpu_phy",
        ]
    ]
    df_lc0 = df[["timestamp", "%cpu_lc0", "cpu_lc0", "phy_cpu", "%cpu_phy", "name"]]
    df_lc1 = df[["timestamp", "%cpu_lc1", "cpu_lc1", "phy_cpu", "%cpu_phy", "name"]]

    df_lc1 = df_lc1.rename(columns={"%cpu_lc1": "%cpu_lc0", "cpu_lc1": "cpu_lc0"})

    df = pd.concat([df_lc0, df_lc1])
    df = df.sort_values(by=["timestamp"])
    df = df.reset_index(drop=True)
    df = df.rename(
        columns={
            "%cpu_lc0": "%cpu",
            "cpu_lc0": "cpu",
            "phy_cpu": "phy",
            "%cpu_phy": "%cpu_phy",
        }
    )

    cpu_df = df.pivot(index=["name", "timestamp"], columns="cpu", values="%cpu")

    phy_df = (
        df.groupby(["name", "timestamp"])
        .agg({"%cpu_phy": "mean"})
        .rename(columns={"%cpu_phy": "physical"})
    )

    df = cpu_df.join(phy_df, on=["name", "timestamp"], how="inner")
    df = df.reset_index()

    return df

In [ ]:
# pivot data into row format
icx_df = pivot_df(icx_df)
clx_df = pivot_df(clx_df)

# calculate remaining utilization
icx_df["baseline"] = icx_df.drop(columns=["name", "timestamp", "physical"]).mean(axis=1)
icx_df["remaining(baseline)"] = 100 - icx_df["baseline"]

clx_df["baseline"] = clx_df.drop(columns=["name", "timestamp", "physical"]).mean(axis=1)
clx_df["remaining(baseline)"] = 100 - clx_df["baseline"]

cols = []
tdf = icx_df.drop(
    columns=["name", "timestamp", "physical", "baseline", "remaining(baseline)"]
)

for a, b in tdf.columns.groupby(tdf.columns % 32).values():
    tdf[f"p{a}"] = tdf[[a, b]].max(axis=1)
    cols.append(f"p{a}")

icx_df["adjusted"] = tdf[cols].mean(axis=1)
icx_df["remaining(rcpu)"] = 100 - icx_df["adjusted"]

cols = []
tdf = clx_df.drop(
    columns=["name", "timestamp", "physical", "baseline", "remaining(baseline)"]
)

for a, b in tdf.columns.groupby(tdf.columns % 40).values():
    tdf[f"p{a}"] = tdf[[a, b]].max(axis=1)
    cols.append(f"p{a}")

clx_df["adjusted"] = tdf[cols].mean(axis=1)

clx_df["remaining(rcpu)"] = 100 - clx_df["adjusted"]

icx_df["remaining(physical)"] = 100 - icx_df["physical"]
clx_df["remaining(physical)"] = 100 - clx_df["physical"]

icx_df = icx_df.reset_index(drop=True)
clx_df = clx_df.reset_index(drop=True)

In [ ]:
from sklearn.model_selection import train_test_split

# split dataset

cols_to_drop_when_x = [
    "name",
    "timestamp",
    "physical",
    "baseline",
    "adjusted",
    "remaining(baseline)",
    "remaining(rcpu)",
    "remaining(physical)",
]

ICX_X = icx_df.drop(columns=cols_to_drop_when_x)
CLX_X = clx_df.drop(columns=cols_to_drop_when_x)

# padding features to 40
ICX_X.columns = [i for i in range(32)]
for i in range(32, 40):
    ICX_X[i] = 0

CLX_X.columns = [i for i in range(40)]

ICX_y = icx_df["remaining(physical)"]
CLX_y = clx_df["remaining(physical)"]

icx_X_train, icx_X_test, icx_y_train, icx_y_test = train_test_split(
    ICX_X, ICX_y, test_size=0.2, random_state=42
)
clx_X_train, clx_X_test, clx_y_train, clx_y_test = train_test_split(
    CLX_X, CLX_y, test_size=0.2, random_state=42
)

In [ ]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

import time

GLOBAL_ERRORS = []
GLOBAL_SPEED = []

RANDOM_INPUT_ARRAY = np.random.random((10000, 40)) * 100 # for speed test

# Evaluation functions
# NOTE: The utilization data in our dataset is already normalized to 100.
#       Thus, the error mentioned in our evaluation is in percentage.
#       We report the raw value in our paper
def eval_on_icx(model, train_on):
    model_name = model.__class__.__name__
    eval_on = "ICX"

    y_pred = model.predict(icx_X_test)

    rmse = np.sqrt(mean_squared_error(icx_y_test, y_pred))
    mae = mean_absolute_error(icx_y_test, y_pred)

    GLOBAL_ERRORS.append(
        {
            "model": model_name,
            "train": train_on,
            "eval": eval_on,
            "rmse": rmse,
            "mae": mae,
        }
    )


def eval_on_clx(model, train_on):
    model_name = model.__class__.__name__
    eval_on = "CLX"

    y_pred = model.predict(clx_X_test)

    rmse = np.sqrt(mean_squared_error(clx_y_test, y_pred))
    mae = mean_absolute_error(clx_y_test, y_pred)

    GLOBAL_ERRORS.append(
        {
            "model": model_name,
            "train": train_on,
            "eval": eval_on,
            "rmse": rmse,
            "mae": mae,
        }
    )


def eval_speed(model):
    model_name = model.__class__.__name__
    times = []

    # warm up
    for _ in range(10):
        model.predict(RANDOM_INPUT_ARRAY)

    for _ in range(20):
        start = time.clock_gettime_ns(time.CLOCK_MONOTONIC)
        model.predict(RANDOM_INPUT_ARRAY) # Shape: (10000, 40)
        end = time.clock_gettime_ns(time.CLOCK_MONOTONIC)

        times.append(end - start)
        time.sleep(0.1)

    GLOBAL_SPEED.append(
        {
            "model": model_name,
            "time": np.mean(times),
            "times": times,
        }
    )

In [ ]:
from sklearn.model_selection import cross_val_score
from hyperopt import STATUS_OK, hp, tpe, Trials, fmin


def icx_objective(space):
    reg = create_model(space) # see function definition below

    score = cross_val_score(
        reg,
        icx_X_train,
        icx_y_train,
        cv=5,
        n_jobs=-1,
        scoring="neg_root_mean_squared_error",
    ).mean()

    return {"loss": -score, "status": STATUS_OK}


def clx_objective(space):
    reg = create_model(space)

    score = cross_val_score(
        reg,
        clx_X_train,
        clx_y_train,
        cv=5,
        n_jobs=-1,
        scoring="neg_root_mean_squared_error",
    ).mean()

    return {"loss": -score, "status": STATUS_OK}


EVAL_TURNS = 20

In [ ]:
set_seed()


def create_model(space):
    return xgb.XGBRegressor(
        random_state=42,
        verbosity=0,
        n_estimators=int(space["n_estimators"]),
        max_depth=int(space["max_depth"]),
        gamma=space["gamma"],
        reg_alpha=int(space["reg_alpha"]),
        min_child_weight=space["min_child_weight"],
        colsample_bytree=space["colsample_bytree"],
    )


space = {
    "max_depth": hp.quniform("max_depth", 3, 18, 1),
    "gamma": hp.uniform("gamma", 1, 9),
    "reg_alpha": hp.quniform("reg_alpha", 40, 180, 1),
    "reg_lambda": hp.uniform("reg_lambda", 0, 1),
    "colsample_bytree": hp.uniform("colsample_bytree", 0.5, 1),
    "min_child_weight": hp.quniform("min_child_weight", 0, 10, 1),
    "n_estimators": hp.uniform("n_estimators", 100, 1000),
}

trials = Trials()
icx_best_params = fmin(
    icx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

trials = Trials()
clx_best_params = fmin(
    clx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

icx_model = create_model(icx_best_params)
clx_model = create_model(clx_best_params)

icx_model.fit(icx_X_train, icx_y_train)
clx_model.fit(clx_X_train, clx_y_train)

eval_on_icx(icx_model, "ICX")
eval_on_clx(clx_model, "CLX")

eval_speed(icx_model)

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

set_seed()


def create_model(space):
    return KNeighborsRegressor(
        n_neighbors=int(space["n_neighbors"]),
        #    weights=space["weights"],
        #    algorithm=space["algorithm"],
        leaf_size=int(space["leaf_size"]),
        p=int(space["p"]),
    )


space = {
    "n_neighbors": hp.uniform("n_neighbors", 1, 100),
    # "weights": hp.choice("weights", ["uniform", "distance"]),
    # "algorithm": hp.choice("algorithm", ["auto", "ball_tree", "kd_tree", "brute"]),
    "leaf_size": hp.uniform("leaf_size", 1, 100),
    "p": hp.uniform("p", 1, 10),
}

trials = Trials()
icx_best_params = fmin(
    icx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

trials = Trials()
clx_best_params = fmin(
    clx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

icx_model = create_model(icx_best_params)
clx_model = create_model(clx_best_params)

icx_model.fit(icx_X_train, icx_y_train)
clx_model.fit(clx_X_train, clx_y_train)

eval_on_icx(icx_model, "ICX")
eval_on_clx(clx_model, "CLX")

eval_speed(icx_model)

In [ ]:
from sklearn.linear_model import LassoCV

set_seed()


def create_model(space):
    return LassoCV(
        cv=int(space["cv"]),
        n_alphas=int(space["n_alphas"]),
        tol=space["tol"],
        max_iter=10000,
    )


space = {
    "cv": hp.uniform("cv", 2, 10),
    "n_alphas": hp.uniform("n_alphas", 1, 100),
    "tol": hp.uniform("tol", 0.0001, 0.001),
    # "max_iter": hp.uniform("max_iter", 100, 3000),
}

trials = Trials()
icx_best_params = fmin(
    icx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

trials = Trials()
clx_best_params = fmin(
    clx_objective, space, algo=tpe.suggest, max_evals=EVAL_TURNS, trials=trials
)

icx_model = create_model(icx_best_params)
clx_model = create_model(clx_best_params)

icx_model.fit(icx_X_train, icx_y_train)
clx_model.fit(clx_X_train, clx_y_train)

eval_on_icx(icx_model, "ICX")

eval_on_clx(clx_model, "CLX")

eval_speed(icx_model)

In [ ]:
from sklearn.svm import SVR

set_seed()


def create_model(space):
    return SVR(
        # kernel=space["kernel"],
        degree=int(space["degree"]),
        gamma=space["gamma"],
        tol=space["tol"],
        C=space["C"],
        epsilon=space["epsilon"],
    )


space = {
    # "kernel": hp.choice("kernel", ["linear", "poly", "rbf", "sigmoid"]),
    "degree": hp.uniform("degree", 1, 10),
    "gamma": hp.uniform("gamma", 0.0001, 1),
    "tol": hp.uniform("tol", 0.0001, 0.001),
    "C": hp.uniform("C", 0.1, 100),
    "epsilon": hp.uniform("epsilon", 0.0001, 0.1),
}

trials = Trials()
icx_best_params = fmin(
    icx_objective, space, algo=tpe.suggest, max_evals=30, trials=trials
)

trials = Trials()
clx_best_params = fmin(
    clx_objective, space, algo=tpe.suggest, max_evals=30, trials=trials
)

icx_model = create_model(icx_best_params)
clx_model = create_model(clx_best_params)

icx_model.fit(icx_X_train, icx_y_train)
clx_model.fit(clx_X_train, clx_y_train)

eval_on_icx(icx_model, "ICX")
eval_on_clx(clx_model, "CLX")

eval_speed(icx_model)

In [ ]:
from sklearn.neural_network import MLPRegressor

set_seed()


def create_model(space):
    return MLPRegressor(
        hidden_layer_sizes=(
            int(space["hidden_layer_sizes1"]),
            int(space["hidden_layer_sizes2"]),
        ),
        learning_rate_init=space["learning_rate"],
        max_iter=10000,
        alpha=space["alpha"],
        momentum=space["momentum"],
        beta_1=space["beta_1"],
        beta_2=space["beta_2"],
        epsilon=space["epsilon"],
        random_state=42,
    )


space = {
    "hidden_layer_sizes1": hp.uniform("hidden_layer_sizes1", 32, 128),
    "hidden_layer_sizes2": hp.uniform("hidden_layer_sizes2", 32, 128),
    "learning_rate": hp.uniform("learning_rate", 0.0001, 0.1),
    # "max_iter": hp.uniform("max_iter", 100, 1000),
    "alpha": hp.uniform("alpha", 0.0001, 0.1),
    "momentum": hp.uniform("momentum", 0.0001, 0.1),
    "beta_1": hp.uniform("beta_1", 0.0001, 0.1),
    "beta_2": hp.uniform("beta_2", 0.0001, 0.1),
    "epsilon": hp.uniform("epsilon", 0.0001, 0.1),
}

trials = Trials()
icx_best_params = fmin(
    icx_objective, space, algo=tpe.suggest, max_evals=30, trials=trials
)

trials = Trials()
clx_best_params = fmin(
    clx_objective, space, algo=tpe.suggest, max_evals=30, trials=trials
)

icx_model = create_model(icx_best_params)
clx_model = create_model(clx_best_params)

icx_model.fit(icx_X_train, icx_y_train)
clx_model.fit(clx_X_train, clx_y_train)

eval_on_icx(icx_model, "ICX")
eval_on_clx(clx_model, "CLX")

eval_speed(icx_model)

In [ ]:
# eval baseline

y_pred = icx_df.iloc[icx_X_test.index]["remaining(baseline)"]

rmse = mean_squared_error(icx_y_test, y_pred, squared=False)
mae = mean_absolute_error(icx_y_test, y_pred)

GLOBAL_ERRORS.append(
    {
        "model": "Baseline",
        "train": "ICX",
        "eval": "ICX",
        "rmse": rmse,
        "mae": mae,
    }
)

y_pred = clx_df.iloc[clx_X_test.index]["remaining(baseline)"]

rmse = mean_squared_error(clx_y_test, y_pred, squared=False)
mae = mean_absolute_error(clx_y_test, y_pred)

GLOBAL_ERRORS.append(
    {
        "model": "Baseline",
        "train": "CLX",
        "eval": "CLX",
        "rmse": rmse,
        "mae": mae,
    }
)

# speed

model_name = "Baseline"
times = []

for i in range(20):
    start = time.clock_gettime_ns(time.CLOCK_MONOTONIC)
    RANDOM_INPUT_ARRAY.mean(axis=1)
    end = time.clock_gettime_ns(time.CLOCK_MONOTONIC)

    times.append(end - start)
    time.sleep(0.5)

GLOBAL_SPEED.append(
    {
        "model": model_name,
        "time": np.mean(times),
        "times": times,
    }
)

In [ ]:
# eval adjusted

y_pred = icx_df.iloc[icx_X_test.index]["remaining(rcpu)"]

rmse = mean_squared_error(icx_y_test, y_pred, squared=False)
mae = mean_absolute_error(icx_y_test, y_pred)

GLOBAL_ERRORS.append(
    {
        "model": "RCPU",
        "train": "ICX",
        "eval": "ICX",
        "rmse": rmse,
        "mae": mae,
    }
)

y_pred = clx_df.iloc[clx_X_test.index]["remaining(rcpu)"]

rmse = mean_squared_error(clx_y_test, y_pred, squared=False)
mae = mean_absolute_error(clx_y_test, y_pred)

GLOBAL_ERRORS.append(
    {
        "model": "RCPU",
        "train": "CLX",
        "eval": "CLX",
        "rmse": rmse,
        "mae": mae,
    }
)

# speed
model_name = "RCPU"
times = []

for i in range(20):
    start = time.clock_gettime_ns(time.CLOCK_MONOTONIC)

    # Take the maximum of every 2 hardware threads
    # The topology follows Intel Ice Lake
    # 0,20 is on the same physical core and 1,21 is on the same physical core ...
    t = np.maximum(RANDOM_INPUT_ARRAY[:, :20], RANDOM_INPUT_ARRAY[:, 20:])
    t = np.mean(t, axis=1)

    end = time.clock_gettime_ns(time.CLOCK_MONOTONIC)

    times.append(end - start)

    time.sleep(0.5)

GLOBAL_SPEED.append(
    {
        "model": model_name,
        "time": np.mean(times),
        "times": times,
    }
)

In [ ]:
errors = pd.DataFrame.from_records(GLOBAL_ERRORS)
errors

In [ ]:
speed = pd.DataFrame.from_records(GLOBAL_SPEED)
speed["speed"] = speed["times"].apply(np.mean)
speed = speed[["model", "speed"]]

baseline = speed.loc[speed.model == "Baseline", "speed"].item()
speed["relative"] = speed["speed"] / baseline
speed = speed.sort_values("relative")
speed